In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit, broadcast, count, avg


24/12/27 16:58:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
 spark = SparkSession.builder \
        .master("local") \
        .appName("hw2") \
        .getOrCreate()
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

24/12/27 17:11:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
    sql_str="WITH yesterday AS ( \
    SELECT * FROM users_cumulated \
    WHERE date = DATE('2023-03-30') \
), \
    today AS ( \
          SELECT user_id, \
                 DATE_TRUNC('day', event_time) AS today_date, \
                 COUNT(1) AS num_events FROM events \
            WHERE DATE_TRUNC('day', event_time) = DATE('2023-03-31') \
            AND user_id IS NOT NULL \
         GROUP BY user_id,  DATE_TRUNC('day', event_time) \
    ) \
--INSERT INTO users_cumulated \
SELECT \
       COALESCE(t.user_id, y.user_id), \
       COALESCE(y.dates_active, \
           ARRAY[]::DATE[]) \
            || CASE WHEN \
                t.user_id IS NOT NULL \
                THEN ARRAY[t.today_date] \
                ELSE ARRAY[]::DATE[] \
                END AS date_list,\
       COALESCE(t.today_date, y.date + Interval '1 day') as date \
FROm yesterday y \
    FULL OUTER JOIN \
    today t ON t.user_id = y.user_id;"
spark.sql(sql_str).show(1)
    

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `events` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 217;
'WithCTE
:- 'CTERelationDef 0, false
:  +- 'SubqueryAlias yesterday
:     +- 'Project [*]
:        +- 'Filter ('date = cast(2023-03-30 as date))
:           +- 'UnresolvedRelation [users_cumulated], [], false
:- 'CTERelationDef 1, false
:  +- 'SubqueryAlias today
:     +- 'Aggregate ['user_id, 'DATE_TRUNC(day, 'event_time)], ['user_id, 'DATE_TRUNC(day, 'event_time) AS today_date#0, count(1) AS num_events#1L]
:        +- 'Filter (('DATE_TRUNC(day, 'event_time) = cast(2023-03-31 as date)) AND isnotnull('user_id))
:           +- 'UnresolvedRelation [events], [], false
+- 'Project [*]
   +- 'SubqueryAlias y
 